#Initialization

###Imports

In [1]:
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
  raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


###Setup input and global variables

In [2]:
configuration = (64, 25, 16)

###Read Datasets

In [3]:
def parse_dataset(filepath):
    """ function used to parse the data of a dataset """

    # open the dataset
    with open(filepath, "rb") as dataset:
        # read the magic number and the number of images
        magic_number, number_of_images = struct.unpack(">II", dataset.read(8))
        # read the number of rows and number of columns per image
        rows, columns = struct.unpack(">II", dataset.read(8))
        # now read the rest of the file using numpy.fromfile()
        images = np.fromfile(dataset, dtype=np.dtype(np.uint8).newbyteorder(">"))
        # reshape so that the final shape is (number_of_images, rows, columns)
        images = images.reshape((number_of_images, rows, columns))

    # return the images
    return images


def parse_labelset(filepath):
    """ function used to parse the data of a labelset """

    # open the file
    with open(filepath, "rb") as labelset:
        # read the magic number and the number of labels
        magic_number, number_of_labels = struct.unpack(">II", labelset.read(8))
        # now read the rest of the file using numpy.fromfile()
        labels = np.fromfile(labelset, dtype=np.dtype(np.uint8).newbyteorder(">"))

    # return the labels
    return labels

In [4]:
# EDIT THE PATHS OF THE DATASETS HERE
train_images_path = os.path.join(".", "drive", "My Drive", "Uni", "Project", "training.dat")
train_labels_path = os.path.join(".", "drive", "My Drive", "Uni", "Project", "training_labels.dat")

# LOAD THE DATASETS HERE
X = parse_dataset(train_images_path)
Y = parse_labelset(train_labels_path)
lb = LabelBinarizer()
Y = lb.fit_transform(Y)

# GET USEFUL VARIABLES
rows = X.shape[1]
columns = X.shape[2]

X = X.reshape(-1, rows, columns, 1)
# normalize
X = X / 255.

# GET VALIDATION DATASET FROM TRAINING SET
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=13, shuffle=True)


# PRINTS TO MAKE SURE
print("X_train.shape = {}".format(X_train.shape))
print("y_train.shape = {}".format(Y_train.shape))
print()
print("X_val.shape = {}".format(X_val.shape))
print("y_val.shape = {}".format(Y_val.shape))

X_train.shape = (51000, 28, 28, 1)
y_train.shape = (51000, 10)

X_val.shape = (9000, 28, 28, 1)
y_val.shape = (9000, 10)


#Classifier Creation

###Create Classifier

In [6]:
from tensorflow.keras.models import load_model

def load_keras_model(model_path):
    """ Function used to load a model from a specific path """

    # just save the model
    return load_model(model_path)

def create_classifier(rows, columns, encoder, units):
    """
    Function that given the encoder part and the dense part of a classifier, creates a "Model"
    (Keras object) that represents a classifier.
    """

    # define the input
    input = Input(shape=(rows, columns, 1))
    x = input

    # pass the input through the encoder
    x = encoder(x)

    x = Flatten()(x)
    # pass then the result through the dense layer
    x = Dense(units=units, activation='relu')(x)
    x = Dense(units=10, activation='softmax')(x)

    # create the model and return it
    classifier = Model(input, x, name="classifier")
    return classifier

###Train

In [7]:
encoder1_path = os.path.join(".", "drive", "My Drive", "Uni", "Project", "e4_7_64_15_16.h5")
encoder2_path = os.path.join(".", "drive", "My Drive", "Uni", "Project", "e3_7_64_15_16_T.h5")
encoder_path = encoder1_path
# encoder_path = encoder2_path
encoder = load_keras_model(encoder_path)

units, epochs, batch_size = configuration

encoder.trainable = False

classifier = create_classifier(rows, columns, encoder, units)
classifier.summary()

callback = ReduceLROnPlateau(monitor="val_loss", factor=1.0/2, patience=4, min_delta=0.005,
                              cooldown=0, min_lr=1e-8, verbose=1)

classifier.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["categorical_crossentropy", "accuracy"])

history = classifier.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                          shuffle=True, validation_data=(X_val, Y_val),
                          callbacks=[callback])

# fine-tune
encoder.trainable = True

history = classifier.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                          shuffle=True, validation_data=(X_val, Y_val),
                          callbacks=[callback])

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder (Functional)         (None, 7, 7, 64)          606528    
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                200768    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 807,946
Trainable params: 201,418
Non-trainable params: 606,528
_________________________________________________________________
Epoch 1/25
3188/3188 [==============================] - 20s 4ms/step - loss: 0.3511 - categorical_crossentropy

#Write the prediction for each image in the dataset to a txt file

In [8]:
def separate_to_clusters(Y):
    clusters = [[] for i in range(10)]

    for i in range(len(Y)):
        clusters[Y[i]].append(i)

    return clusters

def produce_label_file(clusters, file_path):
    file = open(file_path, 'w+')

    for i in range(len(clusters)):
        file.write("CLUSTER-{} {{ size: {}".format(i+1, len(clusters[i])))

        for index in clusters[i]:
            file.write(", {}".format(index))

        file.write("}\n")

    file.close()

In [11]:
Y_prob = classifier.predict(X)
Y_pred = np.round(Y_prob)
Y_unbin = np.argmax(Y_pred, 1)

clusters = separate_to_clusters(Y_unbin)
output_path = os.path.join(".", "drive", "My Drive", "Uni", "Project", "cs64_25_16.txt")
produce_label_file(clusters, output_path)